# Vitamix Forecasting Models

# Global
    * Data Load and Preparation

In [25]:
# Packages
import snowflake.connector
import pandas as pd
import os
import numpy as np
import xlsxwriter
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from tqdm import tqdm

# import the regression module from pycaret   
import pycaret.regression as pycr

# import the time seris module from pycaret

import pycaret.time_series as pycts

# copy and paste in to a new chunk, enter credentials and run to save in environment. Then delete chunk
%env snowflakeuser=<your_snowflake_username>
%env snowflakepass=<your_snowflake_password>

In [3]:
# Query Snowflake

def snowflake_to_pandas(connection_params, query):
    try:
        # Establish a connection to Snowflake
        conn = snowflake.connector.connect(**connection_params)

        # Execute the SQL query and fetch the results into a DataFrame
        df = pd.read_sql_query(query, conn)

        # Close the connection
        conn.close()

        return df
    except Exception as e:
        print(f"Error: {e}")
        return None

# Snowflake connection parameters
connection_params = {
    "user": os.environ['snowflakeuser'],
    "password": os.environ['snowflakepass'],
    "account": "zib52348.us-east-1",
    "role": "ACCOUNTADMIN",
    "warehouse": "REPORTING",
    "database": "ANALYTICS",
    "schema": "FORECASTING",
}

# SQL command 
query = 'SELECT * FROM "ANALYTICS"."FORECASTING"."sales_fcast_combined_v"'

# Call the function to retrieve the data into a Pandas DataFrame
result_df = snowflake_to_pandas(connection_params, query)

if result_df is not None:
    print(result_df.head())  # Display the first few rows of the DataFrame
else:
    print("Failed to retrieve data from Snowflake.")

   DEP_ENT       MONTH     NET_SALES  BUDGET_AMOUNT  FORECAST
0  210_310  2020-09-01  7.449778e+05      1330384.0       NaN
1  210_310  2020-10-01  5.257454e+05       936992.0       NaN
2  210_310  2020-11-01  1.998459e+06      1681287.0       NaN
3  210_310  2020-12-01  4.443028e+05       734573.0       NaN
4  210_310  2021-01-01  3.309476e+05            NaN       NaN


###############
#### NOTES ####
###############

*Change the date below to forecast all of 2023 (8/15/23)

In [36]:
### Data prep ###

result_df["MONTH"] = pd.to_datetime(result_df["MONTH"]) # convert month field to date
#df_all = result_df[result_df['MONTH'] <= pd.Timestamp(2023,6,1)] # we have data through July '23 when we are training these models. 

df_all = result_df[result_df['MONTH'] <= pd.Timestamp(2022,12,1)] # CHANGE FOR 2023

#Some random months will have data that we want to remove (* Want to test without July though)
df_all = df_all[["DEP_ENT", "MONTH", "NET_SALES"]] # select fields of interest
df_all = df_all.sort_values(['DEP_ENT', 'MONTH']) # reorder dataframe

In [37]:
### Spot checks ###

# check the unique time_series. 14 different department-entity combinations
print(df_all['DEP_ENT'].nunique())
print("-----")

# check how many months for each dep-ent. 3 dep-ent do not have all months of data
print(df_all.groupby(['DEP_ENT']).size())
print("-----")

# check for nulls. No nulls
print(df_all.isnull().sum())

14
-----
DEP_ENT
160_155    84
170_155    84
200_155    84
200_310    84
210_155    84
210_165    84
210_310    84
220_155    84
220_310    78
240_155    84
250_155    84
250_165    50
250_310    84
260_155    47
dtype: int64
-----
DEP_ENT      0
MONTH        0
NET_SALES    0
dtype: int64


# Finished with global data work above
__________________________________________________________________________________________________________________________________________________________________________________________________

### 1
# Time Series Forecasting with PyCaret Regression

Pycaret 3.04 regression documentation: https://pycaret.readthedocs.io/en/stable/api/regression.html

In [6]:
### Data Prep for Regression ###

df_all_rg = df_all

# extract month and year from dates
df_all_rg['Month'] = [i.month for i in df_all_rg['MONTH']]
df_all_rg['Year'] = [i.year for i in df_all_rg['MONTH']]

# create a sequence of numbers
df_all_rg['Series'] = np.arange(1,len(df_all_rg)+1)

# Notes on below TS loop
* Manual changes:
  * periods to forecast ahead using the next_dates variable 

In [ ]:
### Regression Loop ###

# Create empty dataframes
all_results_rg = pd.DataFrame()
all_predictions_rg = pd.DataFrame()

# list of each dep-ent
all_ts_rg = df_all_rg['DEP_ENT'].unique()

for i in tqdm(all_ts_rg):
    
    # temp df for a dep-ent
    df_subset = df_all_rg[df_all_rg['DEP_ENT'] == i]
    
    # trim
    df_subset_trim = df_subset[['Series', 'Year', 'Month', 'NET_SALES']] 
    
    # initialize setup
    s = pycr.setup(data = df_subset_trim, target = 'NET_SALES', session_id = 123)
    
    # model training and selection
    best_model = pycr.compare_models()
    
    # capture the compare result grid and store best model in list
    p = pycr.pull().iloc[0:1]
    p['DEP_ENT'] = str(i)
    
    all_results_rg = all_results_rg.append(p)
    
    ### New data to predict ###

    # max date from original dataset
    max_timestamp = df_subset['MONTH'].max()

    # create dataframe for future dates, in this case the rest of 2023
    next_dates = [max_timestamp.replace(day=1) + pd.DateOffset(months=i) for i in range(1, 7)] # Need to change range based on how many periods ahead you want to predict
    new_dates_df = pd.DataFrame({'MONTH': next_dates})
    new_dates_df["MONTH"] = pd.to_datetime(new_dates_df["MONTH"]) 

    # extract month and year from dates
    new_dates_df['Month'] = [i.month for i in new_dates_df['MONTH']]
    new_dates_df['Year'] = [i.year for i in new_dates_df['MONTH']]

    # create a sequence of numbers
    new_dates_df['Series'] = np.arange(1,len(new_dates_df)+1)

    # select cols
    new_dates_df = new_dates_df[['Series', 'Year', 'Month']] 
    
    # predict on new data
    predictions = pycr.predict_model(best_model, data = new_dates_df)
    
    ### Combine predictions in to all_predictions dataframe ###
    
    pred_temp = predictions.reset_index()
    
    pred_comb = pd.concat([pd.DataFrame({'MONTH': next_dates}), pred_temp['prediction_label']], axis = 1)
    pred_comb['DEP_ENT'] = i # add dep
    pred_comb.rename(columns = {'prediction_label':'PRED'}, inplace = True)
    
    # append to master dataframe
    all_predictions_rg = all_predictions_rg.append(pred_comb)

In [12]:
### Write Regression to Excel ###

# Create a Pandas Excel writer using XlsxWriter
rg_writer = pd.ExcelWriter('regression.xlsx', engine='xlsxwriter')

# sheet 1
all_results_rg.to_excel(rg_writer, sheet_name='results', index=False)
# sheet 2
all_predictions_rg.to_excel(rg_writer, sheet_name='predictions', index=False)

# Save Data to File
rg_writer.save()

__________________________________________________________________________________________________________________________________________________________________________________________________

### 2
# Time Series Forecasting with PyCaret Time Series

Pycaret 3.04 time series documentation: https://pycaret.readthedocs.io/en/stable/api/time_series.html

In [38]:
### Data Prep for Time Series Anlysis ###

df_all_ts = df_all

# create time series dataset
df_all_ts = df_all_ts[["DEP_ENT", "MONTH", "NET_SALES"]]

# Notes on below TS loop
* Manual changes:
  * periods to include in the dates dataset created using next_dates
  * forecast horizon (fh) in predict_model function

###############
#### NOTES ####
###############

*When changing dataset to forecast all of 2023, we also need to chnage the dates and fh for thew two notes above (8/15/23)

In [44]:
### Time Series Loop ###

# Create empty dataframes
all_results_ts = pd.DataFrame()
all_predictions_ts = pd.DataFrame()

# list of each dep-ent
all_ts_ts = df_all_ts['DEP_ENT'].unique()

for i in tqdm(all_ts_ts):
    
    # temp df for a dep-ent
    df_subset = df_all_ts[df_all_ts['DEP_ENT'] == i]
    
    # trim
    df_subset_trim = df_subset[['MONTH', 'NET_SALES']] 
    
    # create series
    df_subset_s = df_subset_trim.set_index('MONTH')['NET_SALES']
    
    # set frequency of series
    df_subset_s = df_subset_s.asfreq('MS') 

    # test and train
    ts_train = round(len(df_subset_trim.index) * .8) # 80% train
    ts_test = len(df_subset_trim.index) - round(len(df_subset_trim.index) * .8) # 20% test
    
    # initialize setup
    #s = pycts.setup(df_subset_s, numeric_imputation_target = "mean", fh = ts_test, session_id = 123)
    s = pycts.setup(df_subset_s, numeric_imputation_target = "mean", session_id = 123)

    # model training and selection
    best_model = pycts.compare_models(sort = 'MAPE', exclude = ["naive", "grand_means", "croston", ])
    #best_model = pycts.compare_models(sort = 'MAPE', include = ["arima", "auto_arima", "ets", ])
    
    # plot trained model
    pycts.plot_model(best_model, plot = 'forecast')
    
    # capture the compare result grid and store best model in list
    p = pycts.pull().iloc[0:1]
    p['DEP_ENT'] = str(i)
    
    all_results_ts = all_results_ts.append(p)
    
    ### Predict future periods ###
    
    # max date from original dataset
    max_timestamp = df_subset_trim['MONTH'].max()
    
    # create dataframe for future dates, in this case the rest of 2023. This is not for predicting but to join back up with predictions
    #next_dates = [max_timestamp.replace(day=1) + pd.DateOffset(months=i) for i in range(1, 7)] # Need to change range based on how many periods ahead you want to predict
    
    next_dates = [max_timestamp.replace(day=1) + pd.DateOffset(months=i) for i in range(1, 13)] # CHANGE FOR 2023
    
    new_dates_df = pd.DataFrame({'MONTH': next_dates})
    new_dates_df["MONTH"] = pd.to_datetime(new_dates_df["MONTH"]) 
    
    # predict in unseen future
    #predictions = pycts.predict_model(best_model, fh=6) # Need to change range fh for how many periods ahead you want to predict
    
    predictions = pycts.predict_model(best_model, fh=12) # CHANGE FOR 2023
    
    pred_temp = predictions.reset_index()
    
    pred_comb = pd.concat([pd.DataFrame({'MONTH': next_dates}), pred_temp['y_pred']], axis = 1)
    pred_comb['DEP_ENT'] = i # add dep
    pred_comb.rename(columns = {'y_pred':'PRED'}, inplace = True)
    
    # append to master dataframe
    all_predictions_ts = all_predictions_ts.append(pred_comb)

  0%|          | 0/14 [00:00<?, ?it/s]

,Description,Value
0,session_id,123
1,Target,NET_SALES
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(84, 1)"
5,Transformed data shape,"(84, 1)"
6,Transformed train set shape,"(83, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
polytrend,Polynomial Trend Forecaster,0.1212,0.0959,168497.6577,168497.6577,0.0621,0.0595,0.8167
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,0.1227,0.0971,170614.8954,170614.8954,0.0630,0.0602,0.0767
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,0.1227,0.0971,170614.8954,170614.8954,0.0630,0.0602,0.0900
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,0.1227,0.0971,170614.8954,170614.8954,0.0630,0.0602,0.0767
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,0.1227,0.0971,170614.8954,170614.8954,0.0630,0.0602,0.0800
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,0.1227,0.0971,170614.8954,170614.8954,0.0630,0.0602,0.0800
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,0.1227,0.0971,170614.8954,170614.8954,0.0630,0.0602,0.0400
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,0.1246,0.0986,173369.1781,173369.1781,0.0641,0.0611,0.0767
theta,Theta Forecaster,0.1288,0.1019,178883.3530,178883.3530,0.0655,0.0630,0.5733
exp_smooth,Exponential Smoothing,0.1413,0.1118,196377.5076,196377.5076,0.0721,0.0689,0.6133


  7%|▋         | 1/14 [00:20<04:24, 20.38s/it]

,Description,Value
0,session_id,123
1,Target,NET_SALES
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(84, 1)"
5,Transformed data shape,"(84, 1)"
6,Transformed train set shape,"(83, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


...

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,1.7381,1.3935,371088.1282,371088.1282,0.2334,0.2843,0.0467
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,1.6081,1.2896,343047.8552,343047.8552,0.2729,0.2979,0.0633
snaive,Seasonal Naive Forecaster,1.4484,1.1610,309659.8816,309659.8816,0.3166,0.3066,0.0433
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,1.7577,1.4087,375922.2607,375922.2607,0.3176,0.3394,0.0800
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,1.6323,1.3085,348853.4871,348853.4871,0.3265,0.3309,0.0433
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,1.7113,1.3714,366152.8232,366152.8232,0.3295,0.3409,0.0900
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.6835,1.3495,359848.7241,359848.7241,0.3307,0.3389,0.0900
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.7141,1.3739,366538.2197,366538.2197,0.3375,0.3459,0.0633
theta,Theta Forecaster,1.5932,1.2763,341361.0188,341361.0188,0.3482,0.3380,0.0333
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,1.6946,1.3585,362120.4615,362120.4615,0.3557,0.3520,0.0433


 93%|█████████▎| 13/14 [03:06<00:15, 15.40s/it]

,Description,Value
0,session_id,123
1,Target,NET_SALES
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(47, 1)"
5,Transformed data shape,"(47, 1)"
6,Transformed train set shape,"(46, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
theta,Theta Forecaster,0.6756,0.4434,1321.1390,1321.1390,0.3553,0.3492,0.0367
ets,ETS,0.6845,0.4493,1338.6657,1338.6657,0.3638,0.3535,0.0400
exp_smooth,Exponential Smoothing,0.6858,0.4502,1341.2059,1341.2059,0.3651,0.3540,0.0400
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,0.5907,0.3897,1157.7629,1157.7629,0.3828,0.3103,0.1000
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,0.5780,0.3815,1132.8402,1132.8402,0.3917,0.3060,0.0500
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,0.7165,0.4711,1401.7267,1401.7267,0.4258,0.3601,0.0467
arima,ARIMA,0.7305,0.4805,1429.3332,1429.3332,0.4440,0.3645,0.0567
auto_arima,Auto ARIMA,0.7305,0.4805,1429.3332,1429.3332,0.4440,0.3645,0.1433
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.6984,0.4605,1368.4801,1368.4801,0.4461,0.3550,0.0533
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,0.7498,0.4940,1468.6570,1468.6570,0.4589,0.3778,0.0600


100%|██████████| 14/14 [03:16<00:00, 14.06s/it]


In [45]:
### Write Time Series to Excel ###

# Create a Pandas Excel writer using XlsxWriter
ts_writer = pd.ExcelWriter('time_series.xlsx', engine='xlsxwriter')

# sheet 1
all_results_ts.to_excel(ts_writer, sheet_name='results', index=False)
# sheet 2
all_predictions_ts.to_excel(ts_writer, sheet_name='predictions', index=False)

# Save Data to File
ts_writer.save()